In [ ]:
import TrainModel as Train
import PredictData as Predict

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
import umap
from sklearn.metrics import confusion_matrix, precision_score,recall_score, f1_score, accuracy_score
import umap
import plotly.graph_objects as go
import seaborn as sns


from dataclasses import dataclass, field
from typing import List
from typing import Callable

# Data

In [ ]:
def CheckNan(df):
    nan_columns = df.isna().sum()

    if len(nan_columns[nan_columns> 0]) >0:
        print(f"{len(df)} students")
        raise ValueError(f"These columns contain NaN values this is not going work! \n {nan_columns[nan_columns> 0]}")
    
    #print(f"All good")
    

    
def RemoveEntries(df):

    df2 = df

    _base = len(df)

    #df = df[df["Credits-Y1"] > 0].reset_index(drop=True)
    df = df.dropna(subset=["OverAll"]).reset_index(drop=True)
    df = df.dropna(subset=["B1B2"]).reset_index(drop=True)
    df = df.dropna(subset=["TheRest"]).reset_index(drop=True)   

    df = df.dropna(subset=["TheRest"]).reset_index(drop=True)   

    diff_df = df2.merge(df,on=list(df2.columns), how = 'left', indicator=True)
    missing_rows_df = diff_df[diff_df["_merge"] == "left_only"].drop(columns=["_merge"])





    # print(f"Started with {_base} students")
    # print(f"dropped {_base-len(df)} students")
    # print(missing_rows_df["BSA"].value_counts())
    # print(f"\n =======Remaining========= \n {len(df)}")
    # print(df["BSA"].value_counts())
    return df


def RemoveEntries2(df):

    df2 = df

    _base = len(df)

    df = df[df["BSA"].isin(["PS","NE"])].reset_index(drop=True)
        
    diff_df = df2.merge(df,on=list(df2.columns), how = 'left', indicator=True)
    missing_rows_df = diff_df[diff_df["_merge"] == "left_only"].drop(columns=["_merge"])

    # print(f"Started with {_base} students")
    # print(f"dropped {_base-len(df)} students")
    # print(missing_rows_df["BSA"].value_counts())
    # print(f"\n =======Remaining========= \n {len(df)}")
    # print(df["BSA"].value_counts())
    return df


def DropQuiters(merged_df):
    columns_to_check = [col for col in merged_df.columns if col.startswith(("4","5","6")) and col.endswith("Both_notPresent")]

    rows_to_remove = merged_df[merged_df[columns_to_check].sum(axis=1) >= 3].index
    print(len(rows_to_remove))
    merged_df = merged_df.drop(rows_to_remove)
    return merged_df

def PrepCluster(df, UseNationalities= False, UsePreEducation =True):
    columns_to_drop = []

    columns_to_drop += [col for col in df.columns if col in ['train', 'Year', 'BSA', "Program"]]


    df_Clustering = df.drop(columns=columns_to_drop)

    #Make Nationalities dummies
    if UseNationalities:
        #Create dummy variables
        nationality_dummies = pd.get_dummies(df_Clustering['Nationality'], prefix='Nationality',dtype='int')

        if "Nationality_Niet toeg. Nationaliteit(n/e)" in nationality_dummies.columns:
            nationality_dummies = nationality_dummies.drop(["Nationality_Niet toeg. Nationaliteit(n/e)"], axis=1)
            print('Dropped Nationalities')
            
        # Concatenate the dummy variables with the original DataFrame
        df_Clustering = pd.concat([df_Clustering, nationality_dummies], axis=1)

        Nationalities= ['Nationality_Afrika', 'Nationality_Azie', 'Nationality_EU',
        'Nationality_Europa', 'Nationality_Mid-Zuid-Amerika',
        'Nationality_Nederland', 'Nationality_Noord-Amerika',
        'Nationality_Oceanie', 'Nationality_Onbekend']

        for col_name in Nationalities:
            if col_name not in df_Clustering:
                df_Clustering[col_name] = 0

    if UsePreEducation:
        #Create dummy variables
        preEducation_dummies = pd.get_dummies(df_Clustering['PreEducation'], prefix='PreEducation',dtype='int')

        if "PreEducation_nvt" in preEducation_dummies.columns:
            preEducation_dummies = preEducation_dummies.drop(["PreEducation_nvt"], axis=1)
            #print('Dropped PreEducation_Overig')
            
        if "PreEducation_Overig" in preEducation_dummies.columns:
            preEducation_dummies = preEducation_dummies.drop(["PreEducation_Overig"], axis=1)
            #print('PreEducation_Overig')

        if "PreEducation_Wo" in preEducation_dummies.columns:
            preEducation_dummies = preEducation_dummies.drop(["PreEducation_Wo"], axis=1)
            #print('PreEducation_Wo')         

        # Concatenate the dummy variables with the original DataFrame
        df_Clustering = pd.concat([df_Clustering, preEducation_dummies], axis=1)

        preEducations= ['PreEducation_Buitenlands', 'PreEducation_Hbo', 'PreEducation_Vwo']

        for col_name in preEducations:
            if col_name not in df_Clustering:
                df_Clustering[col_name] = 0

        df_Clustering = df_Clustering[sorted(df_Clustering.columns.difference(preEducations)) + preEducations]

 
    # Drop the original 'Nationality' column
    df_Clustering = df_Clustering.drop(columns=["Nationality","PreEducation"])

    return df_Clustering

def DropGrades(df):
     # Assuming df is your DataFrame
    columns_to_drop = []
    columns_to_drop += [col for col in df.columns if ("grade" in col.lower() )]  
    #columns_to_drop += [col for col in df.columns if col.startswith("2")]  
    df = df.drop(columns=columns_to_drop)

    return df

def PrepB1B2(df):
     # Assuming df is your DataFrame
    columns_to_drop =  [col for col in df.columns if col.startswith(("3","4","5","6"))] 
    columns_to_drop += [col for col in df.columns if col in ["TheRest","OverAll"]]
    columns_to_drop += [col for col in df.columns if (col.startswith(("2")) and col.endswith("Endgrade") )]
    columns_to_drop += [col for col in df.columns if ("grade" in col.lower() )]
    columns_to_drop += [col for col in df.columns if (col.startswith(("2")) and col.endswith("PassedCourse") )]
    columns_to_drop += [col for col in df.columns if (col.startswith(("2")) and col.endswith("Both_notPresent") )]

    
    #columns_to_drop += [col for col in df.columns if col.startswith("2")]  
    df = df.drop(columns=columns_to_drop)
    return df


def MakeSimple(df):
     # Assuming df is your DataFrame
    columns_to_drop = []
    columns_to_drop += [col for col in df.columns if (col.startswith(("1")) and col.endswith("FirstTryPassed") )]
    columns_to_drop += [col for col in df.columns if (col.startswith(("1")) and col.endswith("FirstTryPresent") )]
    columns_to_drop += [col for col in df.columns if (col.startswith(("1")) and col.endswith("Both_notPresent") )]
    
    columns_to_drop += [col for col in df.columns if (col.startswith(("2")) and col.endswith("FirstTryPresent") )]
    
    #columns_to_drop += [col for col in df.columns if col.startswith("2")]  
    df = df.drop(columns=columns_to_drop)
    return df

def MakeRoel(df):
     # Assuming df is your DataFrame
    columns_to_drop = []
    columns_to_drop += [col for col in df.columns if col.endswith("FirstTryPassed")]
    columns_to_drop += [col for col in df.columns if col.endswith("FirstTryPresent")]
    columns_to_drop += [col for col in df.columns if col.endswith("PassedCourse")]   
    columns_to_drop += [col for col in df.columns if (col.startswith(("1")) and col.endswith("Both_notPresent") )]
    
    columns_to_drop += [col for col in df.columns if (col.startswith(("2")) and col.endswith("FirstTryPresent") )]

    #columns_to_drop += [col for col in df.columns if col.startswith("2")]  
    df = df.drop(columns=columns_to_drop)
    return df

def MakeRoelV2(df):
     # Assuming df is your DataFrame
    columns_to_drop = []
    columns_to_drop += [col for col in df.columns if col.endswith("FirstTryPassed")]
    columns_to_drop += [col for col in df.columns if col.endswith("FirstTryPresent")]
    columns_to_drop += [col for col in df.columns if col.endswith("PassedCourse")]   
    columns_to_drop += [col for col in df.columns if (col.startswith(("1")) and col.endswith("Both_notPresent") )]
    
    columns_to_drop += [col for col in df.columns if (col.startswith(("2")) and col.endswith("FirstTryPresent") )]


    columns_to_drop += [col for col in df.columns if (col.startswith(("2")) and col.endswith("FirstTryPresent") )]
    columns_to_drop += [col for col in df.columns if (col.startswith(("2")) and col.endswith("FirstTryPresent") )]

    columns_to_drop += [col for col in df.columns if (col.startswith(("PreEducation")))]
    columns_to_drop += [col for col in df.columns if (col.startswith(("Roel_Resit_Factor")))]

    columns_to_drop += [col for col in df.columns if (col.startswith(("Roel_Resit_Factor")))]
    #columns_to_drop += [col for col in df.columns if col.startswith("2")]  
    df = df.drop(columns=columns_to_drop)
    return df



def RemoveRoel(df):
    columns_to_drop = []
    columns_to_drop += [col for col in df.columns if col.endswith("Roel_Resit_Factor")]
    columns_to_drop += [col for col in df.columns if col.endswith("Total_absent")]
    #columns_to_drop += [col for col in df.columns if col.startswith("2")]  
    df = df.drop(columns=columns_to_drop)
    return df





In [ ]:
@dataclass
class Model:
    name: str
    modelMode: str
    preprocessingMode: str
    smoteMode: str

    traindata: pd.DataFrame
    train: Callable[[pd.DataFrame, bool],Train.TrainedModel] = field(init=False,repr=False)
    trainedmodel: Train.TrainedModel

    def __init__(self,modelMode,preprocessingMode,smoteMode,traindata,trainmethod) -> None:

        self.modelMode = modelMode
        self.preprocessingMode = preprocessingMode
        self.smoteMode = smoteMode

        self.traindata = traindata
        self.train = trainmethod

        self.name = preprocessingMode + "_" + modelMode + "_" + smoteMode 


In [ ]:
def GetTheModels(modelList: List[Model],df_cluster,preproName,smoteName):
    print(f"GetTheModels")

    svm = Model("SVM",preproName,smoteName,PrepB1B2(df_cluster),Train.SVM_Default)
    svm.trainedmodel = svm.train(svm.traindata,False)
    modelList.append(svm)
    print(f"SVM ")

    nnA = Model("NN",preproName,smoteName,PrepB1B2(df_cluster),Train.NeuralNetwork_Default)
    nnA.trainedmodel = nnA.train(nnA.traindata,False)
    modelList.append(nnA)
    print(f"NN ")
    # xgA = Model("xG",preproName,smoteName,PrepB1B2(df_cluster),Train.XgBoost)
    # xgA.trainedmodel = xgA.train(xgA.traindata,False)
    # modelList.append(xgA)
    # print(f"xG ")


    rF = Model("rF",preproName,smoteName,PrepB1B2(df_cluster),Train.RandomForest)
    rF.trainedmodel = rF.train(rF.traindata,False)
    modelList.append(rF)
    print(f"rF ")

    # GBM = Model("GBM",preproName,smoteName,PrepB1B2(df_cluster),Train.GBM)
    # GBM.trainedmodel = GBM.train(GBM.traindata,False)
    # modelList.append(GBM)
    # print(f"GBM")


    KNeighbor = Model("KNeighbor",preproName,smoteName,PrepB1B2(df_cluster),Train.KNeighbor)
    KNeighbor.trainedmodel = KNeighbor.train(KNeighbor.traindata,False)
    modelList.append(KNeighbor)
    print(f"KNeighbor")

    NaiveBayes = Model("NaiveBayes",preproName,smoteName,PrepB1B2(df_cluster),Train.NaiveBayes)
    NaiveBayes.trainedmodel = NaiveBayes.train(NaiveBayes.traindata,False)
    modelList.append(NaiveBayes)
    print(f"NaiveBayes")

    LRegression = Model("LRegression",preproName,smoteName,PrepB1B2(df_cluster),Train.LRegression)
    LRegression.trainedmodel = LRegression.train(LRegression.traindata,False)
    modelList.append(LRegression)
    print(f"LRegression")

    # decisionTreeClassifier = Model("Tree",preproName,smoteName,PrepB1B2(df_cluster),Train.decisionTreeClassifier)
    # decisionTreeClassifier.trainedmodel = decisionTreeClassifier.train(decisionTreeClassifier.traindata,False)
    # modelList.append(decisionTreeClassifier)
    # print(f"decisionTreeClassifier")

    
    # lgbBig = Model("lgbBig",preproName,smoteName,PrepB1B2(df_cluster),Train.lgbBig)
    # lgbBig.trainedmodel = lgbBig.train(lgbBig.traindata,False)
    # modelList.append(lgbBig)
    # print(f"lgbBig")


    return modelList

In [ ]:
def run(i):
    print(f"Program {i}")
    df = pd.concat([
        pd.read_excel(f'PrepareDatasets/Train_Program{i}_22_23.xlsx', sheet_name=0),
        pd.read_excel(f'PrepareDatasets/Train_Program{i}_21_22.xlsx', sheet_name=0),
        ],ignore_index=True)

    df = RemoveEntries(df)
    df['Gender'] = df['Gender'].apply(lambda x: 1 if x == 'F' else 0)

    
    modelList: List[Model] = []


    dfTrueBSA = RemoveEntries2(df)
    dfTrueBSA_cluser = PrepCluster(dfTrueBSA)
    dfTrueBSA_cluser = DropGrades(dfTrueBSA_cluser)
    CheckNan(dfTrueBSA_cluser)
    dfTrueBSA_cluser["BSA"] = (dfTrueBSA_cluser["Credits-Y1"] > 42).astype(int)
    dfTrueBSA_cluser = RemoveRoel(dfTrueBSA_cluser)   

    GetTheModels(modelList,dfTrueBSA_cluser,"High","False")



    try:
        dfTrueBSA_cluser_smoteB1B2 = SmoteB1B2(dfTrueBSA_cluser)
        GetTheModels(modelList,dfTrueBSA_cluser_smoteB1B2,"High","True")  
    except:
        pass

    # try:
    #     dfTrueBSA_cluser_smoteB1B2 = SmoteSmart(dfTrueBSA_cluser)
    #     GetTheModels(modelList,dfTrueBSA_cluser_smoteB1B2,"High","SmoteSmart")  
    # except:
    #     pass



    #simple
    dfSimple = RemoveEntries2(df)
    dfSimple_cluser = PrepCluster(dfSimple)
    dfSimple_cluser = DropGrades(dfSimple_cluser)
    CheckNan(dfSimple_cluser)
    dfSimple_cluser["BSA"] = (dfSimple_cluser["Credits-Y1"] > 42).astype(int)
    dfSimple_cluser = MakeSimple(dfSimple_cluser)

    GetTheModels(modelList,dfSimple_cluser,"Medium","False") 


    try:
        dfSimple_cluser_smoteB1B2 = SmoteB1B2(dfSimple_cluser)
        GetTheModels(modelList,dfSimple_cluser_smoteB1B2,"Medium","True")   
    except:
        pass





    #Roel
    dfRoel_cluser = PrepCluster(RemoveEntries2(df))
    dfRoel_cluser = DropGrades(dfRoel_cluser)
    CheckNan(dfRoel_cluser)
    dfRoel_cluser["BSA"] = (dfRoel_cluser["Credits-Y1"] > 42).astype(int)
    dfRoel_cluser = MakeSimple(dfRoel_cluser)
    dfRoel_cluser = MakeRoel(dfRoel_cluser)
    GetTheModels(modelList,dfRoel_cluser,"Low","False")  



    try:
        dfRoel_cluser_smoteB1B2 = SmoteB1B2Credits(dfRoel_cluser)
        GetTheModels(modelList,dfRoel_cluser_smoteB1B2,"Low","True")    
    except:
        pass


    dfRoel_cluserV2 = MakeRoelV2(dfRoel_cluser)
    GetTheModels(modelList,dfRoel_cluserV2,"Roel","False")  



    try:
        dfRoel_cluser_smoteB1B2 = SmoteB1B2Credits(dfRoel_cluserV2)
        GetTheModels(modelList,dfRoel_cluser_smoteB1B2,"Roel","True")    
    except:
        pass





    df_test = pd.concat([
    pd.read_excel(f'PrepareDatasets/Test_Program{i}_21_22.xlsx', sheet_name=0),
    pd.read_excel(f'PrepareDatasets/Test_Program{i}_22_23.xlsx', sheet_name=0),   
     ],ignore_index=True)
    df_test['Gender'] = df_test['Gender'].apply(lambda x: 1 if x == 'F' else 0)



    df_test_cluser = PrepCluster(df_test)
    df_test_cluser = PrepB1B2(df_test_cluser)
    df_test_cluser["BSA"] = (df_test_cluser["Credits-Y1"] >= 42).astype(int)
    df_test = Predict.PrepareDF(df_test)
    Predict.AddDefaultRuleToDF(df_test)


    for model in modelList:
        Predict.AddModelToDF(model.trainedmodel,model.name,df_test,df_test_cluser)

    

    df2324 = pd.read_excel(f'PrepareDatasets/Pred_Program{i}_23_24.xlsx', sheet_name=0)
    df2324['Gender'] = df2324['Gender'].apply(lambda x: 1 if x == 'F' else 0)
    df2324_cluser = PrepCluster(df2324)
    df2324_cluser = PrepB1B2(df2324_cluser)

    df2324 = Predict.PrepareDF(df2324)
    Predict.AddDefaultRuleToDF(df2324)


    for model in modelList:
        Predict.AddModelToDF(model.trainedmodel,model.name,df2324,df2324_cluser)

    


    return df_test, df2324

# SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE

def Smote(df_cluser):
    smote = SMOTE()

    X_resampled, y_resampled = smote.fit_resample(df_cluser.drop(columns=["Credits-Y1", "BSA"]),df_cluser["Credits-Y1"])

    df_cluser_smote = pd.DataFrame(X_resampled)
    df_cluser_smote["Credits-Y1"] = y_resampled
    df_cluser_smote["BSA"] = df_cluser_smote["Credits-Y1"] >= 42#y_resampled
    df_cluser_smote = df_cluser_smote.astype(int)



    return df_cluser_smote


def SmoteB1B2(df_cluser):

    smote = SMOTE()

    X_resampled, y_resampled = smote.fit_resample(df_cluser.drop(columns=["Credits-Y1", "BSA","Crd-B1B2"]),df_cluser["Crd-B1B2"])

    df_cluser_smote = pd.DataFrame(X_resampled)


    df_cluser_smote["Crd-B1B2"] = y_resampled
    
    passed_course_columns = [col for col in df_cluser_smote.columns if col.endswith("PassedCourse")]

    passed_course_df = df_cluser_smote[passed_course_columns]

    df_cluser_smote["Credits-Y1"] = passed_course_df.sum(axis=1) * 6
    df_cluser_smote["BSA"] = df_cluser_smote["Credits-Y1"] >= 42#y_resampled
    df_cluser_smote = df_cluser_smote.astype(int)
    return df_cluser_smote






def SmoteB1B2Credits(df_cluser):
    smote = SMOTE()

    X_resampled, y_resampled = smote.fit_resample(df_cluser.drop(columns=["BSA"]),df_cluser["Crd-B1B2"])

    df_cluser_smote = pd.DataFrame(X_resampled)
    df_cluser_smote["BSA"] = df_cluser_smote["Credits-Y1"] >= 42#y_resampled
    df_cluser_smote = df_cluser_smote.astype(int)

    return df_cluser_smote



def SmoteSmart(df_cluser):

    smote = SMOTE()

    df_cluser["MinortiyClass"] = df_cluser["Crd-B1B2"].apply(lambda x: 1 if x == 12 else 0)

    X_resampled, y_resampled = smote.fit_resample(df_cluser.drop(columns=["Credits-Y1", "BSA","MinortiyClass"]),df_cluser["MinortiyClass"])

    df_cluser_smote = pd.DataFrame(X_resampled)


    #df_cluser_smote["Crd-B1B2"] = y_resampled
    
    passed_course_columns = [col for col in df_cluser_smote.columns if col.endswith("PassedCourse")]

    passed_course_df = df_cluser_smote[passed_course_columns]

    df_cluser_smote["Credits-Y1"] = passed_course_df.sum(axis=1) * 6
    df_cluser_smote["BSA"] = df_cluser_smote["Credits-Y1"] >= 42#y_resampled
    df_cluser_smote = df_cluser_smote.astype(int)
    return df_cluser_smote



# def ADASYNB1B2(df_cluser):

#     smote = ADASYN()

#     X_resampled, y_resampled = smote.fit_resample(df_cluser.drop(columns=["Credits-Y1", "BSA","Crd-B1B2"]),df_cluser["Crd-B1B2"])

#     df_cluser_smote = pd.DataFrame(X_resampled)


#     df_cluser_smote["Crd-B1B2"] = y_resampled
    
#     passed_course_columns = [col for col in df_cluser_smote.columns if col.endswith("PassedCourse")]

#     passed_course_df = df_cluser_smote[passed_course_columns]

#     df_cluser_smote["Credits-Y1"] = passed_course_df.sum(axis=1) * 6
#     df_cluser_smote["BSA"] = df_cluser_smote["Credits-Y1"] >= 42#y_resampled
#     df_cluser_smote = df_cluser_smote.astype(int)
#     return df_cluser_smote



# def SmoteSmart(df_cluser):
#     df_temp = df_cluser[(df_cluser['Crd-B1B2'] >= 6) & (df_cluser['Crd-B1B2'] <= 18)]



#     plt.rcParams["figure.figsize"] = (6,6)
#     unique_elements, count_elements = np.unique(df_temp["BSA"], return_counts=True)
#     fig1,ax1 = plt.subplots()
#     ax1.pie(count_elements,labels=unique_elements, autopct = "%1.1f%%", startangle=90,textprops={"fontsize":18})



#     smote = SMOTE()

#     X_resampled, y_resampled = smote.fit_resample(df_temp.drop(columns=["Credits-Y1", "BSA"]),df_temp["BSA"])

#     df_cluser_smote = pd.DataFrame(X_resampled)
#     #df_resampled_clustering["Credits-Y1"] = y_resampled
#     df_cluser_smote["BSA"] = y_resampled#df_resampled_clustering["Credits-Y1"] >= 42
#     df_cluser_smote = df_cluser_smote.astype(int)



#     df_cluser_smote_smart = pd.concat([
#         df_cluser,
#         df_cluser_smote,
#         ],ignore_index=True)
    
    
#     return df_cluser_smote_smart







# Model Testing

In [ ]:
dftestlist = [4,
1,
2,
3,

]

In [ ]:
writer = pd.ExcelWriter("Test.xlsx",engine="xlsxwriter")
writer_Pred = pd.ExcelWriter("Predict.xlsx",engine="xlsxwriter")
df_modeleval = pd.DataFrame(columns=["Program","Preprocessing","Model","Smote","Filter","Accuracy","Precision","Recall","F1_score"])


for i in dftestlist:
    df_test, df2324 = run(i)
    round_columns = ["OverAll",	"B1B2",	"TheRest"]

    for col in round_columns:
        df_test[col] = df_test[col].apply(lambda x: round(x,2))
        
    for col in round_columns:
        df2324[col] = df2324[col].apply(lambda x: round(x,2))


    hash_columns = [col for col in df_test.columns if col.startswith("#")]
    
    roelresultaten = pd.read_excel(f'Z:/RLust/Data/MAARTEN 2.0.xlsx', sheet_name=i-1)

    df_test["#@Roel"] = roelresultaten[f"binary_predict"].dropna().astype(int)
    df_test.to_excel(writer, sheet_name=f"Program{i}",index=False)

    df2324.to_excel(writer_Pred, sheet_name=f"Program{i}",index=False)

    workbook = writer.book
    worksheet = writer.sheets[f"Program{i}"]

    green_format = workbook.add_format({"bg_color":"#81d41a"})
    Red_format = workbook.add_format({"bg_color":"#c7785d"})
    black_format = workbook.add_format({"bg_color":"#000000"})

    for col_num, col_name in enumerate(df_test.columns,1):  
        if col_name.startswith("#"): #or col_name =="BSA_Compare"
            for row_num, value in enumerate(df_test[col_name],1):
                if value == -1:
                    worksheet.write(row_num,col_num-1,"",black_format)
                elif value == df_test["BSA_Compare"][row_num-1]:
                    worksheet.write(row_num,col_num-1,value,green_format)
                else:
                    worksheet.write(row_num,col_num-1,value,Red_format)



    dfTestList = []
    dfTestList.append(df_test)
    dfTestList.append(df_test[(df_test["Crd-B1B2"] == 12)])
    dfTestList.append(df_test[(df_test["BSA"] != "DI") & (df_test["BSA"] != "STGA")])
    dfTestList.append(df_test[(df_test["Crd-B1B2"] == 12) & (df_test["BSA"] != "DI") & (df_test["BSA"] != "STGA")])

    dfTestList_Names = ["ALL", "B1B2-12EC","NO_DI_STGA","NO_DI_STGA-B1B2-12EC"]

    combined = list(zip(dfTestList_Names,dfTestList))



    #Sheet model eval
    for col in df_test.columns[df_test.columns.get_loc("BSA_Compare") +1:]:

        for _name, _df in combined:

            model_name = col
            pred_labels = df_test.loc[_df.index][col]
            true_labels = df_test.loc[_df.index]["BSA_Compare"]
            
            filtered_lists = [(x,y) for x,y in zip(pred_labels,true_labels) if x != -1]

            pred_labels,true_labels = zip(*filtered_lists)

            split_lst = model_name.split("_")

            if model_name == "#Default":
                split_lst= ["","Default",""]

            if model_name == "#@Roel":
                split_lst= ["@Roel","LRegression","False"]

            new_entries = {
                "Program": f"Program{i}",
                "Preprocessing":split_lst[0],
                "Model":split_lst[1],
                "Smote":split_lst[2],
                "Filter":_name,     
                "Accuracy":round(accuracy_score(true_labels, pred_labels),5) ,
                "Precision":round(precision_score(true_labels, pred_labels),5) ,
                "Recall":round(recall_score(true_labels, pred_labels),5),
                "F1_score": round(f1_score(true_labels, pred_labels),5)}

            df_modeleval = pd.concat([df_modeleval,pd.DataFrame([new_entries])], ignore_index=True)


df_modeleval.to_excel(writer, sheet_name="ModelEval",index=False)


writer.close()

writer_Pred.close()

# cell_range = f"{chr(64+ col_num)}2:{chr(64+ col_num)}{len(df_test)+1}"

# color_scale_format ={
#     "type"
# }

In [ ]:
writer_Pred.close()


# df_test.loc[df_test[col] != df_test["BSA_Compare"]]